In [1]:
%reload_ext autoreload
%autoreload 2

from _header import *

Using device:	 mps


---
## Load Models and apply LoRA

In [ ]:
# PLM

plm_tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path=BASE_MODEL_PLM,
    do_lower_case=False,
    use_fast=True,
    legacy=False,
)

plm_model, plm_loading_info = T5EncoderModel.from_pretrained(
    pretrained_model_name_or_path=BASE_MODEL_PLM,
    output_loading_info=True,
    # device_map=device,
    # load_in_8bit=False,
    # custom_dropout_rate=0.1,
)

plm_lora_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q", "k", "v", "o"],
    bias="none",
)

plm_model = peft.get_peft_model(plm_model, plm_lora_config)
plm_model.print_trainable_parameters()


# LLM

llm_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=BASE_MODEL_LLM
)

llm_model, llm_loading_info = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_LLM,
    # device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    output_loading_info=True,
)

llm_lora_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
    bias="none",
)

llm_model = peft.get_peft_model(llm_model, llm_lora_config)
llm_model.print_trainable_parameters()

trainable params: 3,932,160 || all params: 1,212,073,984 || trainable%: 0.3244


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,456,448 || all params: 3,825,536,000 || trainable%: 0.1165


---
## Train

In [ ]:
ds_dummy = {
    "MLEVPVWIPILAFAVGLGLGLLIPHLQKPFQRF": "This protein is involved in membrane transport.",
    "MSLEQKKGADIISKILQIQNSIGKTTSPSTLKT": "This enzyme catalyzes the hydrolysis of ATP.",
    "MKMKQQGLVADLLPNIRVMKTFGHFVFNYYNDN": "This transcription factor regulates gene expression."
}

---
## Inference

In [ ]:
with torch.no_grad():
    protein_features = plm_base_model()['last_hidden_state']
    language_features = llm_base_model()['last_hidden_state']